# Matching Algorithm Model Comparison

Here you will find a comprehensive comparison using our existing match data and testing on various models along with annotation describing each model. The next chunk of code here is just importing various libraries as well as loading in our data.

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import itertools

from numpy.random import randn
from numpy.matlib import repmat

from scipy.stats import norm
from scipy.optimize import fmin
from scipy.special import erf

from patsy import dmatrices

from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LassoCV, RidgeClassifierCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.feature_selection import chi2
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
job_seekers = pd.read_csv('/home/nolski/Downloads/test.csv')
job_seekers = job_seekers[job_seekers['year2'] == '1']
job_seekers = job_seekers[job_seekers['employed_6_week'].isin(['0', '1'])]

In [3]:
nationality = ['syrian', 'jordanian']
gender = ['male', 'female']
secondary = ['0', '1']
work = [0, 1]
cols = ['nationality', 'gender', 'above_secondary_edu', 'ever_employed']
lists = [nationality, gender, secondary, work]

stratum = pd.DataFrame(list(itertools.product(*lists)), columns=cols)
stratum.to_csv('/home/nolski/Downloads/stratum.csv')

In [4]:
def set_strata(j):
    test = [j['nationality'], j['gender'], j['above_secondary_edu'], j['ever_employed']]
    return stratum[(stratum['nationality'] == test[0]) & 
            (stratum['gender'] == test[1]) & 
            (stratum['above_secondary_edu'] == test[2]) & 
            (stratum['ever_employed'] == test[3])].index[0]
    

In [5]:
def has_intervention(j, i):
    return 1 if j['actual_intervention_received'] == i else 0

In [6]:
new = pd.DataFrame()
new['covar1'] = job_seekers.apply(set_strata, axis=1)
new['treatment1'] = job_seekers.apply(has_intervention, args=('cash',), axis=1)
new['treatment2'] = job_seekers.apply(has_intervention, args=('information',), axis=1)
new['treatment3'] = job_seekers.apply(has_intervention, args=('psychological',), axis=1)
new['outcome'] = job_seekers['employed_6_week']

In [7]:
new['outcome'].replace(['0'], 'FALSE', inplace=True)
new['outcome'].replace(['1'], 'TRUE', inplace=True)

In [8]:
new.to_csv('/home/nolski/Downloads/thompson-input.csv', index=False)